In [1]:
import theano
import theano.tensor as T
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))

In [2]:
logistic = theano.function([x], s)

In [3]:
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

###### Computing more than one thing at the same time!

In [4]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a, b], [diff, abs_diff, diff_squared])
f([[23, 10]], [[30, 10]])

[array([[-7.,  0.]]), array([[ 7.,  0.]]), array([[ 49.,   0.]])]

###### Setting the default inputs!

In [6]:
from theano import In
x, y = T.dscalars('x', 'y')
z = x + y

In [7]:
f = theano.function([x, In(y, value = 1)], z)

In [8]:
f(33)

array(34.0)

In [9]:
f(33, 2)

array(35.0)

In [10]:
# setting the name parameter of In by
# In(w, value = 2, name = 'new_name')
# this overrides the variable's name during the function
# call, we donot create a new variable!

Using shared variables!

In [11]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates = [(state, state + inc)])

**Shared variables** can be used between multiple functions. Unlike other objects(dmatrices, dscalars), shared variables have an internal value that defines the value taken by the symbolic variable in all the functions that use it. 

**updates** must be supplied with a pair of form ``(shared variable, new expression)`` or a `dict` with **keys as shared variables and values as new expressions**  

In [12]:
print(state.get_value())

0


In [13]:
accumulator(1)

array(0)

In [14]:
print(state.get_value())

1


In [15]:
accumulator(300)

array(1)

In [16]:
print(state.get_value())

301


In [17]:
# Resetting the value of the shared varible 
state.set_value(-1)
accumulator(3)
print(state.get_value())

2


In [18]:
#### Using more than one function with the shared varibale
decrementor = theano.function([inc], state, updates = [(state, state - inc)])

In [19]:
decrementor(2)
print(state.get_value())

0


#### Skipping a shared variable!

In [20]:
fn_of_state = state * 2 + inc
# Here we didn't mention a prefix for foo(iscalar, dscalar),
# we transferred the datatype of the 
foo = T.scalar(dtype = state.dtype)
## this will output function of state 
skip_shared = theano.function([inc, foo], fn_of_state, givens = [(state, foo)] )

In [21]:
skip_shared(1, 3)

array(7)

In [22]:
print(state.get_value())

0


In [23]:
print(foo)

<TensorType(int64, scalar)>


#### Copying functions
Used to make same functions but with different shared variables.

In [24]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates = [(state, state + inc)])

In [25]:
accumulator(10)
print(state.get_value())

10


In [26]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state: new_state})

In [27]:
new_accumulator(100)

[array(0)]

In [28]:
print(new_state.get_value())

100


In [29]:
null_accumulator = accumulator.copy(delete_updates = True)

In [30]:
null_accumulator(191010101)
print(state.get_value())

10


##### Using Random Numbers

In [31]:
from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed = 234)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = theano.function([], rv_u)
g = theano.function([], rv_n, no_default_updates = True)
#Since we are not updating rv_n
nearly_zeros = theano.function([], rv_u + rv_u - 2 *rv_u)

In [32]:
f_val0 = f()
f_val1 = f()
print(f_val0)
print(f_val1)

[[ 0.12672381  0.97091597]
 [ 0.13989098  0.88754825]]
[[ 0.31971415  0.47584377]
 [ 0.24129163  0.42046081]]


In [33]:
g_val0 = g()
g_val1 = g()
print(g_val0, g_val1)

[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]] [[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]


In [34]:
nearly_zeros()

array([[ 0.,  0.],
       [ 0.,  0.]])

**Remark: ** The random varible is drawn at most once during a single function call and execution. So, the `nearly_zeros()` return 0 even when rv_u is used three times.